In [ ]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML
font = {'size'   : 15}
matplotlib.rc('font', **font)

In this notebook we solve the KdV equation

$$
    u_t = uu_x + u_{xxx}
$$

with a wavepacket initial condition.

In [ ]:
def rk3(u,xi,rhs,dt,epsilon=1.0,linear=False):
    y2 = u + dt*rhs(u,xi,epsilon=epsilon, linear=linear)
    y3 = 0.75*u + 0.25*(y2 + dt*rhs(y2,xi,epsilon=epsilon, linear=linear))
    u_new = 1./3 * u + 2./3 * (y3 + dt*rhs(y3,xi,epsilon=epsilon, linear=linear))
    return u_new


def rhs(u, xi, epsilon=1.0, linear=False):
    uhat = np.fft.fft(u)
    if linear:
        return epsilon*np.real(np.fft.ifft(-1j*xi**3*uhat))
    else:
        return u*np.real(np.fft.ifft(1j*xi*uhat)) + epsilon*np.real(np.fft.ifft(-1j*xi**3*uhat))
    
def solve_KdV(u0,tmax=1.,m=256,epsilon=1.0, ylims=(-100,300), linear=False):
    """Solve the KdV equation using Fourier spectral collocation in space
       and SSPRK3 in time, on the domain (-pi, pi).  The input u0 should be a function.
    """
    # Grid
    L = 2*np.pi
    x = np.arange(-m/2,m/2)*(L/m)
    xi = np.fft.fftfreq(m)*m*2*np.pi/L

    dt = 1.73/((m/2)**3)
    u = u0(x)
    uhat2 = np.abs(np.fft.fft(u))

    num_plots = 400
    nplt = np.floor((tmax/num_plots)/dt)
    nmax = int(round(tmax/dt))

    fig = plt.figure(figsize=(12,8))
    axes = fig.add_subplot(211)
    axes2 = fig.add_subplot(212)
    line, = axes.plot(x,u,lw=3)
    line2, = axes2.semilogy(xi,uhat2**2)
    xi_max = np.max(np.abs(xi))
    axes.set_xlabel(r'$x$',fontsize=30)
    axes2.set_xlabel(r'$\xi$',fontsize=30)
    plt.tight_layout()
    plt.close()
    
    frames = [u.copy()]
    tt = [0]
    uuhat = [uhat2]

    for n in range(1,nmax+1):
        u_new = rk3(u,xi,rhs,dt,linear=linear)

        u = u_new.copy()
        t = n*dt
        # Plotting
        if np.mod(n,nplt) == 0:
            frames.append(u.copy())
            tt.append(t)
            uhat2 = np.abs(np.fft.fft(u))
            uuhat.append(uhat2)
            
    def plot_frame(i):
        line.set_data(x,frames[i])
        power_spectrum = np.abs(uuhat[i])**2
        line2.set_data(np.sort(xi),power_spectrum[np.argsort(xi)])
        axes.set_title('t= %.2e' % tt[i])
        axes.set_xlim((-np.pi,np.pi))
        axes.set_ylim(ylims)

    anim = matplotlib.animation.FuncAnimation(fig, plot_frame,
                                       frames=len(frames), interval=100)

    return HTML(anim.to_jshtml())

## Linear KdV solution
By looking at the motion of individual peaks versus the envelope, you can see the difference between the phase velocity $k_0^2$ and the group velocity $3k_0^2$.

The lower plot shows the power spectrum of the solution.  Note that for the linearized KdV equation (as for any linear wave equation), the power spectrum is constant in time.

In [ ]:
def u0(x):
    return np.cos(80*x) * np.exp(-10*x**2)
solve_KdV(u0, m=512, tmax=0.00005, ylims=(-1,1), linear=True)

Next we consider the nonlinear KdV equation.  The idea here is that the envelope of the wavepacket should approximately obey the NLS equation, but I haven't finished developing this.

In [ ]:
def u0(x):
    return 1000*np.cos(80*x) * np.exp(-10*x**2)
solve_KdV(u0, m=512, tmax=0.0005, ylims=(-1000,1000), epsilon=0.01)

In [ ]:
def u0(x):
    return 1000*np.cos(80*x) * np.exp(-10*x**2)
solve_KdV(u0, m=1024, tmax=0.0005, ylims=(-1000,1000))